In [1]:
from cashe import get_cashed_input
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
import time
from src.model_train import encode_label
import numpy as np
train_columns = ['SERVICE_DESCRIPTION', 'SERVICE_TYPE','UNIT','UNIT_TYPE','TIMES','PER',
                 'OASIS_IOS_DESCRIPTION', 'PROVIDER_DEPARTMENT','ICD10','OUTCOME']

In [2]:
df_original = get_cashed_input()
preprocessing = MergedDataPreprocessing(df_original)

df_original = preprocessing.columns_prep()
df_train, df_test = preprocessing.train_test_split_time()

E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column]= df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange']= df['PatientAgeRange'].replace(age_encoding)


In [6]:
df_train[5:10]

,REQUEST_DATE,VISIT_NO,VISIT_DATE,EMERGENCY_INDICATOR,PROVIDER_DEPARTMENT_CODE,PROVIDER_DEPARTMENT,DOCTOR_SPECIALTY_CODE,DOCTOR_CODE,PATIENT_NO,EPISODE_NO,...,UNIT_PRICE_STOCKED_UOM,UNIT_PRICE_NET,DISCOUNT_PERCENTAGE,NOTES,OUTCOME,APPROVED_QUNATITY,Episode_Key,ICD10,Diagnosis_Key,PatientAgeRange
398452,2024-01-01 12:38:36,1343731-2,2023-12-03 00:00:00,0,25,RADIOGRAPHIC INVESTIGATIONS,18.06,9638,1180507,2,...,4.72,3.49,7.43,Rejected for the reason Sorry you are not an a...,REJECTED,0,1_1343731-2,N80.1,1_1111228814,5
398673,2024-01-02 01:04:02,1335736-8,2023-12-07 10:29:11,0,18,OBSTERTRIC/GYNEACOLOGIC SERVIC,10.00,56,1173039,8,...,11.98,11.38,5.01,CV-1-1Services performed by a non-network p...,REJECTED,0,1_1335736-8,Encounter for cesarean delivery without indica...,1_1111231440,6
398668,2024-01-02 01:04:02,1335736-8,2023-12-07 10:29:11,0,18,OBSTERTRIC/GYNEACOLOGIC SERVIC,10.00,56,1173039,8,...,340,272,20,CV-1-1Services performed by a non-network p...,REJECTED,0,1_1335736-8,Encounter for cesarean delivery without indica...,1_1111231440,6
398675,2024-01-02 01:04:02,1335736-8,2023-12-07 10:29:11,0,18,OBSTERTRIC/GYNEACOLOGIC SERVIC,10.00,56,1173039,8,...,500,500,0,CV-1-1Services performed by a non-network p...,REJECTED,0,1_1335736-8,Encounter for cesarean delivery without indica...,1_1111231440,6
398674,2024-01-02 01:04:02,1335736-8,2023-12-07 10:29:11,0,18,OBSTERTRIC/GYNEACOLOGIC SERVIC,10.00,56,1173039,8,...,15700,13345,15,CV-1-1Services performed by a non-network p...,REJECTED,0,1_1335736-8,Encounter for cesarean delivery without indica...,1_1111231440,6


In [7]:
X_train = df_train[train_columns[:-1]];  y_train = df_train[train_columns[-1]].values
X_test = df_test[train_columns[:-1]];  y_test = df_test[train_columns[-1]].values

y_train = encode_label(y_train); y_test = encode_label(y_test)

In [8]:
X= df_train[df_train.columns]
X

,REQUEST_DATE,VISIT_NO,VISIT_DATE,EMERGENCY_INDICATOR,PROVIDER_DEPARTMENT_CODE,PROVIDER_DEPARTMENT,DOCTOR_SPECIALTY_CODE,DOCTOR_CODE,PATIENT_NO,EPISODE_NO,...,UNIT_PRICE_STOCKED_UOM,UNIT_PRICE_NET,DISCOUNT_PERCENTAGE,NOTES,OUTCOME,APPROVED_QUNATITY,Episode_Key,ICD10,Diagnosis_Key,PatientAgeRange
398448,2024-01-01 12:33:25,1346536-2,2023-12-07,0,855,CARDIOTHORACIC,08.25,8455,1183018,2,...,120,96,0,,APPROVED,1.0,1_1346536-2,J93,1_1111225808,6
398449,2024-01-01 12:38:12,1343530-4,2023-12-06,0,4,DENTAL SERVICES,22.00,7352,1180321,4,...,120,120,0,Rejected for the reason Sorry you are not an a...,REJECTED,0,1_1343530-4,K02,1_1111225570,1
398453,2024-01-01 12:38:36,1343731-2,2023-12-03,0,25,RADIOGRAPHIC INVESTIGATIONS,18.06,9638,1180507,2,...,1791.67,1154.17,30,Rejected for the reason Sorry you are not an a...,REJECTED,0,1_1343731-2,N80.1,1_1111228814,5
398454,2024-01-01 12:38:36,1343731-2,2023-12-03,0,25,RADIOGRAPHIC INVESTIGATIONS,18.06,9638,1180507,2,...,1.69,1.25,7.69,Rejected for the reason Sorry you are not an a...,REJECTED,0,1_1343731-2,N80.1,1_1111228814,5
398450,2024-01-01 12:38:36,1343731-2,2023-12-03,0,25,RADIOGRAPHIC INVESTIGATIONS,18.06,9638,1180507,2,...,16.13,11.29,30.01,Rejected for the reason Sorry you are not an a...,REJECTED,0,1_1343731-2,N80.1,1_1111228814,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301019,2024-04-06 13:14:09,1360424-3,2024-03-04,0,29,CARDIOLOGY SERVICES,08.02,3396,1195919,3,...,14.85,.4703333333333333333333333333333333333,4.98,,APPROVED,1,1_1360424-3,I21,1_111225233,7
301018,2024-04-06 13:14:09,1360424-3,2024-03-04,0,29,CARDIOLOGY SERVICES,08.02,3396,1195919,3,...,56.45,1.787666666666666666666666666666666666,5.01,,APPROVED,1,1_1360424-3,I21,1_111225233,7
301017,2024-04-06 13:14:09,1360424-3,2024-03-04,0,29,CARDIOLOGY SERVICES,08.02,3396,1195919,3,...,86.9,2.751666666666666666666666666666666666,5.01,BE-1-1,PARTIAL,1,1_1360424-3,I21,1_111225233,7
301022,2024-04-06 13:14:09,1360424-3,2024-03-04,0,29,CARDIOLOGY SERVICES,08.02,3396,1195919,3,...,6.35,.067,5.04,BE-1-1,PARTIAL,1,1_1360424-3,I21,1_111225233,7


In [9]:
lstm_embedding = LSTMEmbedding()
X_train_processed = lstm_embedding.preprocess_data(X_train) ## 16-dim embedding
X_train_processed

array([[  36,   16,    1, ...,    0,    0,    0],
       [  16,   91,  123, ...,    0,    0,    0],
       [1618,  164,   41, ...,    0,    0,    0],
       ...,
       [ 402,  149,  396, ...,   31,    1,  265],
       [ 344,   82,    9, ...,  344,   82,    9],
       [  59,   16,    1, ...,    0,    0,    0]])

In [10]:
print('Training Model')
lstm_embedding.train_model(X_train_processed, y_train,epochs=3,batch_size=32)

print('Evaluating Model')
accuracy = lstm_embedding.evaluate_model(X_test, y_test)

print('Model Accuracy on test data: ',accuracy)

Training Model
Epoch 1/3
5175/5175 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.5613 - loss: 0.6784
Epoch 2/3
5175/5175 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.5799 - loss: 0.6677
Epoch 3/3
5175/5175 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.5929 - loss: 0.6592
Evaluating Model
1294/1294 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5693 - loss: 0.6726
Model Accuracy on test data:  0.6825442314147949


In [11]:
lstm_embedding.input_shape

(16, 1)

In [12]:
X_test_vectorized = lstm_embedding.embedding_vector(X_test,reload_model=False)
X_test_vectorized

array([[  29,    3,    4, ...,   18,    1,  244],
       [   3,    4,  241, ...,   18,    1,  244],
       [  36,   16,    1, ...,    0,    0,    0],
       ...,
       [2968,  889,  559, ...,    0,    0,    0],
       [  11,   19,   74, ...,   18,    1, 4878],
       [ 403,   33,  103, ...,    0,    0,    0]])

In [13]:
start_time = time.time()
lstm_encoder = LSTMEmbedding()
lstm_encoder.load_model()
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:10],reload_model=True)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time per example: {:.2f} seconds".format(execution_time))

Execution time per example: 0.29 seconds


In [14]:
X_test_vectorized

array([[374, 268, 302, 276,   2,  62, 445,  54, 374, 268, 302, 276,   2,
          0,   0,   0],
       [272, 185, 648, 321, 455, 253, 272, 185, 648, 321,   0,   0,   0,
          0,   0,   0],
       [ 77,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 29, 267, 733,  64,  29, 267, 733,  64,  73,  22, 253,   0,   0,
          0,   0,   0],
       [196,   6,  36,  43,  62,  43,  62, 196,   6,  36,  43,  73,  22,
        253,   0,   0],
       [ 35,   2,  68,  62,  43,  62,  67,  45,   6,   2,  35,   2,  68,
         73,  22, 253],
       [830,   5,  75,  62, 245, 830,   5,  75,  62, 245,  73,  22, 253,
          0,   0,   0],
       [ 73,  22, 253,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [515,   8, 515,   8,  73,  22, 253,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 70,   5, 123,  70,   5, 123,  73,  22, 253,   0,   0,   0,   0,
          0,   0,   0]])

In [15]:
def vectorize(COUNT_VALS):
      preds = lstm_encoder.embedding_vector(X_test[:COUNT_VALS],reload_model=True)


def calculate_excecution():
  exs = []
  COUNT_VALS = 50
  for i in range(10):
    start_time = time.time()

    vectorize(COUNT_VALS)

    end_time = time.time()

    execution_time = end_time - start_time
    exs.append(execution_time)

  average = np.array(exs).mean()
  return average/COUNT_VALS

calculate_excecution()

0.0011286411285400392